In [1]:
import os
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from models.mesh_classifier import ClassifierModel

from data.data import ClassificationData, collate_fn
import utils.util as util

In [2]:
dataroot = 'datasets/human_class' 
phase = 'train'
export_folder = 'export'
checkpoints_dir = 'checkpoints'
name = 'classification'
ninput_edges = 40000
num_aug = 1

batch_size = 16
num_workers = 4

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
train_dataset = ClassificationData(dataroot=dataroot, phase='train', export_folder=export_folder, device=device, ninput_edges=ninput_edges, num_aug=num_aug)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    collate_fn=collate_fn
)

nclasses = train_dataset.get_nclasses()   
ninput_channels = train_dataset.get_ninput_channels()

print('nclasses', nclasses)
print('ninput_channels', ninput_channels)

test_dataset = ClassificationData(dataroot=dataroot, phase='test', export_folder=export_folder, device=device, ninput_edges=ninput_edges, num_aug=num_aug)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)

loaded mean / std from cache
nclasses 2
ninput_channels 5
loaded mean / std from cache


In [5]:
from models import networks

init_type = 'normal'
init_gain = 0.02

net = networks.MeshClassifier(ninput_channels, nclasses, ninput_edges)
net = networks.init_weights(net, init_type, init_gain)
net.to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=0.005, betas=(0.9, 0.999))

In [6]:
for epoch in range(0,10):
  
  net.train()
  for i, data in enumerate(train_dataloader):
    
    mesh = data['mesh']
    labels = torch.from_numpy(data['label']).long().to(device)
    edge_features = torch.from_numpy(data['edge_features']).float().to(device).requires_grad_(True)
    
    optimizer.zero_grad()
  
    out = net(edge_features, mesh)
    labels = labels.repeat(out.shape[0])
    loss = criterion(out, labels)
    loss.backward()
    optimizer.step()
    
    print(f'[Epoch {epoch+1} - {i+1}/{len(train_dataloader)}] loss: {loss.item()}')
  
  # net.eval()
  # with torch.no_grad():
  #   for i, data in enumerate(test_dataloader):
  #     mesh = data['mesh']
  #     labels = torch.from_numpy(data['label']).long().to(device)
  #     edge_features = torch.from_numpy(data['edge_features']).float().to(device).requires_grad_(False)
      
  #     out = net(edge_features, mesh)
  #     labels = labels.repeat(out.shape[0])
  #     print(f'out: {out}')  
  #     print(f'labels: {labels}')

[Epoch 1 - 1/42] loss: 0.6418492794036865
[Epoch 1 - 2/42] loss: 0.2878410220146179
[Epoch 1 - 3/42] loss: 4.382134914398193
[Epoch 1 - 4/42] loss: 2.612250566482544
[Epoch 1 - 5/42] loss: 1.2724547386169434
[Epoch 1 - 6/42] loss: 0.5803623199462891
[Epoch 1 - 7/42] loss: 0.6348061561584473
[Epoch 1 - 8/42] loss: 0.6428232789039612
[Epoch 1 - 9/42] loss: 0.644263505935669
[Epoch 1 - 10/42] loss: 0.744667649269104
[Epoch 1 - 11/42] loss: 0.7442841529846191
[Epoch 1 - 12/42] loss: 0.645368754863739
[Epoch 1 - 13/42] loss: 0.6455217003822327
[Epoch 1 - 14/42] loss: 0.74359530210495
[Epoch 1 - 15/42] loss: 0.7433652877807617
[Epoch 1 - 16/42] loss: 0.6459108591079712
[Epoch 1 - 17/42] loss: 0.645939826965332
[Epoch 1 - 18/42] loss: 0.7428432106971741
[Epoch 1 - 19/42] loss: 0.6454477906227112
[Epoch 1 - 20/42] loss: 0.743071436882019
[Epoch 1 - 21/42] loss: 0.742644727230072
[Epoch 1 - 22/42] loss: 0.7417670488357544
[Epoch 1 - 23/42] loss: 0.647036075592041
[Epoch 1 - 24/42] loss: 0.73976